# AutoGen tööriista kasutamise näide


## Impordi Vajalikud Paketid


In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## Kliendi loomine

Selles näites kasutame [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) platvormi, et pääseda ligi LLM-ile.

`model` on määratletud kui `gpt-4o-mini`. Proovi mudelit muuta mõnele teisele GitHub Models turul saadaval olevale mudelile, et näha erinevaid tulemusi.

Kiireks testiks käivitame lihtsa küsimuse - `Mis on Prantsusmaa pealinn`.


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Funktsioonide määratlemine

Selles näites anname agendile juurdepääsu tööriistale, mis on funktsioon koos nimekirjaga saadaolevatest puhkuse sihtkohtadest ja nende saadavusest.

Võite ette kujutada, et see oleks olukord, kus reisikonsultandil on näiteks juurdepääs reiside andmebaasile.

Kui näidet läbi töötate, proovige julgelt määratleda uusi funktsioone ja tööriistu, mida agent saab kasutada.


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## Funktsiooni tööriista määratlemine
Selleks, et agent saaks kasutada `vacation_destinations` kui `FunctionTool`, peame selle sellisena määratlema.

Samuti anname tööriistale kirjelduse, mis aitab agendil mõista, milleks seda tööriista kasutatakse seoses kasutaja poolt esitatud ülesandega.


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## Agendi määratlemine

Nüüd saame allolevas koodis agendi luua. Määratleme `system_message`, et anda agendile juhised, kuidas aidata kasutajatel leida puhkuse sihtkohti.

Samuti määrame parameetri `reflect_on_tool_use` väärtuseks true. See võimaldab LLM-il tööriista kasutamise vastust võtta ja vastuse loomulikus keeles edastada.

Saate parameetri väärtuseks määrata false, et näha erinevust.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## Agendi käivitamine

Nüüd saame käivitada agendi, alustades kasutaja esialgsest sõnumist, milles palutakse korraldada reis Tokyosse.

Saate muuta selle linna sihtkohta, et näha, kuidas agent reageerib linna saadavusele.


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**Lahtiütlus**:  
See dokument on tõlgitud AI tõlketeenuse [Co-op Translator](https://github.com/Azure/co-op-translator) abil. Kuigi püüame tagada täpsust, palume arvestada, et automaatsed tõlked võivad sisaldada vigu või ebatäpsusi. Algne dokument selle algses keeles tuleks pidada autoriteetseks allikaks. Olulise teabe puhul soovitame kasutada professionaalset inimtõlget. Me ei vastuta selle tõlke kasutamisest tulenevate arusaamatuste või valesti tõlgenduste eest.
